In [1]:
if True:
    import os
    os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
    execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
    import os
    import sys
else:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
    sys.path.insert(0, '/usr/lib/spark/python/')
    sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
    sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext, HiveContext
    
    try: sc = SparkContext()
    except: None    
    sqlc = SQLContext(sc)
    spark = sqlc

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.0.2
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType
import pyspark.sql.functions as F

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
#spark = sqlc

In [6]:
import seaborn as sns

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [7]:
import numpy as np
import pandas as pd
import time
import json

In [8]:
from operator import add
from pyspark.sql.types import *

In [9]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from datetime import datetime

In [16]:
def convert_uir_tupple(line):
    r = line.split(',')
    return (int(r[0]), int(r[1]), float(r[2]))

def load_train():
    raw = sc.textFile("lab_12/input/train.csv")\
            .filter(lambda x: not x.startswith("userId"))\
            .map(convert_uir_tupple)
    return raw.toDF(schema=StructType([StructField("userId",  IntegerType()),
                                           StructField("movieId", IntegerType()),
                                           StructField("rating",  FloatType())])) 
    

In [17]:
train = load_train()

In [18]:
train.count()

10531564

In [13]:
# rerun eval notebook with full data set

In [14]:
strain = train

# Baseline with Basic predictiors approach

In [15]:
smooth_buser = 5
smooth_bitem = 10

In [16]:
avg_rating = strain.agg({"rating":"mean"}).collect()[0]

In [17]:
avg_rating = avg_rating[0]
print 'avg rating', avg_rating

avg rating 3.52186384662


In [18]:
buser = strain.groupBy("userId").agg(F.sum(F.col("rating")).alias("sum_ratings"), 
                                     F.count(F.col("rating")).alias("n_user_ratings"))

In [19]:
buser.show(2)

+------+-----------+--------------+
|userId|sum_ratings|n_user_ratings|
+------+-----------+--------------+
| 54431|        5.0|             1|
| 54631|       27.0|             8|
+------+-----------+--------------+
only showing top 2 rows



In [20]:
x = buser.select("userId", "n_user_ratings", 
                 ((F.col("sum_ratings") - F.lit(avg_rating)*F.col("n_user_ratings"))  / (F.col("n_user_ratings") + F.lit(smooth_buser))).alias("bpu"))

In [21]:
x.show(2)

+------+--------------+-------------------+
|userId|n_user_ratings|                bpu|
+------+--------------+-------------------+
|107231|            11|-0.4212813945512499|
|107431|            19| 0.4618577880924999|
+------+--------------+-------------------+
only showing top 2 rows



In [22]:
buser = x

In [23]:
strain.registerTempTable("strain")

In [24]:
buser.registerTempTable("buser")

In [26]:
bitem = spark.sql("""
            select
                t.movieId, sum(t.rating - %g - bu.bpu) as sum_r_normalised, count(*) n_item_ratings
            from strain t
            join buser bu on t.userId = bu.userId 
            group by t.movieId
""" % (avg_rating,))

In [27]:
bitem.show(2)

+-------+-----------------+--------------+
|movieId| sum_r_normalised|n_item_ratings|
+-------+-----------------+--------------+
|  11631|167.9514203739218|          1120|
|   7831|4427.922156680536|         10731|
+-------+-----------------+--------------+
only showing top 2 rows



In [28]:
x = bitem.select("movieId", "n_item_ratings", 
                 ( F.col("sum_r_normalised")  / (F.col("n_item_ratings") + F.lit(smooth_bitem))).alias("bpi"))

In [29]:
x.show(2)

+-------+--------------+--------------------+
|movieId|n_item_ratings|                 bpi|
+-------+--------------+--------------------+
|  10031|          2904|-0.07798649355444924|
|   7831|         10731| 0.41224487074579075|
+-------+--------------+--------------------+
only showing top 2 rows



In [30]:
bitem = x

In [31]:
bitem.registerTempTable("bitem")

## Build prediction for strain set

In [34]:
q = spark.sql("""
        select 
            t.*,
            bu.n_user_ratings,
            bi.n_item_ratings,
            ({avg_rating} + bu.bpu + bi.bpi) as bp_predicted_rating,
            rating - ({avg_rating} + bu.bpu + bi.bpi) as bp_err
        from strain t
        join buser bu on t.userId = bu.userId 
        join bitem bi on t.movieId = bi.movieId
""".format(avg_rating=avg_rating))

In [35]:
strain_predictions = q

In [36]:
strain_predictions.show(2)

+------+-------+------+--------------+--------------+-------------------+-------------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|bp_predicted_rating|             bp_err|
+------+-------+------+--------------+--------------+-------------------+-------------------+
|112036|     31|   4.0|          1376|           161| 3.3182498501676094| 0.6817501498323906|
| 14649|     31|   3.0|          2262|           161|  3.409838526906103|-0.4098385269061029|
+------+-------+------+--------------+--------------+-------------------+-------------------+
only showing top 2 rows



In [37]:
def calc_rmse(df, prediction_col_name):
    mse = df.select(F.pow(F.col(prediction_col_name) - F.col("rating"), F.lit(2)).alias("err"))\
            .agg({"err":"mean"}).collect()[0][0]
    return np.sqrt(mse)    

In [39]:
rmse = calc_rmse(strain_predictions, "bp_predicted_rating")

In [40]:
print 'strain RMSE', rmse

strain RMSE 0.867715175772


In [41]:
def cut_1to5(x):
    if x < 1.:
        return 1.0
    if x > 5:
        return 5.0
    return x

udf_cut_1to5 = F.udf(cut_1to5, FloatType())

In [42]:
#q = strain_predictions.withColumn("predicted_rating", udf_cut_1to5(F.col("predicted_rating")))

In [43]:
#q.show(2)

In [44]:
#print 'strain RMSE with cut1t5', calc_rmse(q)

## Cache results

In [45]:
#strain_predictions.coalesce(8).write.parquet("lab_12/cache/full_train_bp",
#                                             mode='overwrite')

## Build prediction for test set

In [19]:
def convert_ui_tupple(line):
    r = line.split(',')
    return (int(r[0]), int(r[1]))
    
def load_test():    
    raw = sc.textFile("lab_12/input/test.csv")\
            .filter(lambda x: not x.startswith("userId"))\
            .map(convert_ui_tupple)
    return raw.toDF(schema=StructType([StructField("userId",  IntegerType()),
                                       StructField("movieId", IntegerType())]))

In [20]:
test = load_test()

In [21]:
test.count()

10531564

In [59]:
stest = test

In [60]:
stest.registerTempTable("stest")

In [64]:
q = spark.sql("""
        select 
            t.*,
            bu.n_user_ratings,
            bi.n_item_ratings,
            (%g + coalesce(bu.bpu, 0.0) + coalesce(bi.bpi, 0.0)) as bp_predicted_rating
        from stest t
        left join buser bu on t.userId = bu.userId 
        left join bitem bi on t.movieId = bi.movieId
""" % (avg_rating, ))

In [65]:
q.show(2)

+------+-------+--------------+--------------+-------------------+
|userId|movieId|n_user_ratings|n_item_ratings|bp_predicted_rating|
+------+-------+--------------+--------------+-------------------+
|223036|     31|          1376|           161|  3.548151868862599|
| 72254|     31|             9|           161|  2.908756988943684|
+------+-------+--------------+--------------+-------------------+
only showing top 2 rows



In [66]:
stest_predictions = q

## Cache bp predictions on full test

In [67]:
#stest_predictions.coalesce(8).write.parquet("lab_12/cache/full_test_bp",
#                                             mode='overwrite')

In [ ]:
def cut_1to5(x):
    if x < 1.:
        return 1.0
    if x > 5:
        return 5.0
    return x

udf_cut_1to5 = F.udf(cut_1to5, FloatType())


In [ ]:
qs = q.select("userId", "movieId", udf_cut_1to5(F.col("predicted_rating")).alias("rating")).orderBy("userId", "movieId")

In [ ]:
res = qs.toPandas()

In [ ]:
res.to_csv("lab_12.csv", index=False)